In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat'))
lons = np.asarray(getvar(ctl,'lon'))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure'))
p_adj = np.asarray(getvar(adj,'pressure'))

### Height
z_ctl = np.asarray(getvar(ctl,'z'))
z850 = interplevel(z_ctl, p_ctl, 850)
z700 = interplevel(z_ctl, p_ctl, 700)
z600 = interplevel(z_ctl, p_ctl, 600)
z500 = interplevel(z_ctl, p_ctl, 500)
z400 = interplevel(z_ctl, p_ctl, 400)
z300 = interplevel(z_ctl, p_ctl, 300)

### U
u_ctl = np.asarray(getvar(ctl,'ua'))
u_adj = np.asarray(getvar(adj,'ua'))
du850 = interplevel(u_adj, p_adj, 850) - interplevel(u_ctl, p_ctl, 850)
du700 = interplevel(u_adj, p_adj, 700) - interplevel(u_ctl, p_ctl, 700)
du600 = interplevel(u_adj, p_adj, 600) - interplevel(u_ctl, p_ctl, 600)
du500 = interplevel(u_adj, p_adj, 500) - interplevel(u_ctl, p_ctl, 500)
du400 = interplevel(u_adj, p_adj, 400) - interplevel(u_ctl, p_ctl, 400)
du300 = interplevel(u_adj, p_adj, 300) - interplevel(u_ctl, p_ctl, 300)

### V
v_ctl = np.asarray(getvar(ctl,'va'))
v_adj = np.asarray(getvar(adj,'va'))
dv850 = interplevel(v_adj, p_adj, 850) - interplevel(v_ctl, p_ctl, 850)
dv700 = interplevel(v_adj, p_adj, 700) - interplevel(v_ctl, p_ctl, 700)
dv600 = interplevel(v_adj, p_adj, 600) - interplevel(v_ctl, p_ctl, 600)
dv500 = interplevel(v_adj, p_adj, 500) - interplevel(v_ctl, p_ctl, 500)
dv400 = interplevel(v_adj, p_adj, 400) - interplevel(v_ctl, p_ctl, 400)
dv300 = interplevel(v_adj, p_adj, 300) - interplevel(v_ctl, p_ctl, 300)

### T
t_ctl = np.asarray(getvar(ctl,'THM')) + TR
t_adj = np.asarray(getvar(adj,'THM')) + TR
tk_ctl = t_ctl/((P0/p_ctl)**(RD/CP))
tk_adj = t_adj/((P0/p_adj)**(RD/CP))
dtk850 = interplevel(tk_adj, p_adj, 850) - interplevel(tk_ctl, p_ctl, 850)
dtk700 = interplevel(tk_adj, p_adj, 700) - interplevel(tk_ctl, p_ctl, 700)
dtk600 = interplevel(tk_adj, p_adj, 600) - interplevel(tk_ctl, p_ctl, 600)
dtk500 = interplevel(tk_adj, p_adj, 500) - interplevel(tk_ctl, p_ctl, 500)
dtk400 = interplevel(tk_adj, p_adj, 400) - interplevel(tk_ctl, p_ctl, 400)
dtk300 = interplevel(tk_adj, p_adj, 300) - interplevel(tk_ctl, p_ctl, 300)

### DTE
dt850 = np.sqrt((du850**2 + dv850**2 + (CP/TR)*dtk850**2)/2)
dt700 = np.sqrt((du700**2 + dv700**2 + (CP/TR)*dtk700**2)/2)
dt600 = np.sqrt((du600**2 + dv600**2 + (CP/TR)*dtk600**2)/2)
dt500 = np.sqrt((du500**2 + dv500**2 + (CP/TR)*dtk500**2)/2)
dt400 = np.sqrt((du400**2 + dv400**2 + (CP/TR)*dtk400**2)/2)
dt300 = np.sqrt((du300**2 + dv300**2 + (CP/TR)*dtk300**2)/2)



In [ ]:
### Contour interval
zlvls = np.arange(0,2.2,0.2)

cmaplist = plt.cm.turbo(np.linspace(0.2, 1, len(zlvls) + 1))
cmaplist[0] = (1.0,1.0,1.0,1.0)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('mcm',cmaplist, len(zlvls)+1)
cmap.set_over('k')

### Plot subdomain
min_lat = 15
max_lat = 62
min_lon = -130
max_lon = -70

### Target region
min_lat1 = 25
max_lat1 = 40
min_lon1 = -107
max_lon1 = -92

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],['bottomleft','bottommiddle','bottomright'],['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(8,5), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels = False

cs1 = axd['topleft'].contour(lons, lats, smooth2d(z300,1)/10.,
                  levels=np.arange(0,1500,25),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,25),fontsize=6,colors='k')
im1 = axd['topleft'].contourf(lons, lats, dt300, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

axd['topleft'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Title
axd['topleft'].set_title('300 hPa')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
#############

gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(z400,1)/10.,
                  levels=np.arange(0,1500,25),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,25),fontsize=6,colors='k')
im1 = axd['topmiddle'].contourf(lons, lats, dt400, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['topmiddle'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

axd['topmiddle'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Title
axd['topmiddle'].set_title('400 hPa')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topright'].contour(lons, lats, smooth2d(z500,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topright'].contourf(lons, lats, dt500, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

axd['topright'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())

# Title
axd['topright'].set_title('500 hPa')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(z600,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomleft'].contourf(lons, lats, dt600, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

axd['bottomleft'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())


# Title
axd['bottomleft'].set_title('600 hPa')
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
#############

gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(z700,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottommiddle'].contourf(lons, lats, dt700, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

axd['bottommiddle'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())


# Title
axd['bottommiddle'].set_title('700 hPa')
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
#############

gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(z850,1)/10.,
                  levels=np.arange(0,1500,5),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,5),fontsize=6,colors='k')
im1 = axd['bottomright'].contourf(lons, lats, dt850, levels=zlvls,
            cmap = cmap, extend='max',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=0.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.5,color='k',alpha=0.5)

axd['bottomright'].plot([min_lon1, min_lon1, max_lon1, max_lon1, min_lon1], 
                    [min_lat1, max_lat1, max_lat1, min_lat1, min_lat1], 
         color='magenta', linewidth=1.5, marker='None',
         transform=crs.PlateCarree())


# Title
axd['bottomright'].set_title('850 hPa')
axd['bottomright'].text(0.0, 1.0, '(f)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig04_wrf_perts.pdf',bbox_inches='tight')
plt.show()
